# Retrieval Augmented Generation (RAG) with OpenAI

**Converted to use OpenAI API (from Azure OpenAI)**

## Setup Instructions:

1. **Get an OpenAI API key:**
   - Go to https://platform.openai.com/
   - Sign up or log in
   - Go to API Keys section
   - Create a new API key
   - Add $5-10 in credits to your account

2. **Set your API key:**
   - Replace `"your-api-key-here"` with your actual API key
   - **OR** create a `.env` file with: `OPENAI_API_KEY=your-key-here`

3. **Run the cells in order!**

---


In [ ]:
!pip install requests faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━ 2.9/3.5 MB 34.2 kB/s eta 0:00:17

In [2]:
import faiss
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Sample documents (small knowledge base)
documents = [
    "Python is a programming language.",
    "The capital of France is Paris.",
    "Azure is a cloud computing platform by Microsoft.",
    "OpenAI provides powerful language models for various tasks.",
    "The Great Wall of China is a historic landmark."
]

# Convert documents into TF-IDF vectors
vectorizer = TfidfVectorizer()
document_vectors = vectorizer.fit_transform(documents).toarray()

# Index the document vectors using FAISS (for fast retrieval)
index = faiss.IndexFlatL2(document_vectors.shape[1])
index.add(np.array(document_vectors).astype(np.float32))

# Function to retrieve the most relevant document
def retrieve(query, top_n=1):
    query_vector = vectorizer.transform([query]).toarray()
    distances, indices = index.search(query_vector.astype(np.float32), top_n)
    return [documents[i] for i in indices[0]]


In [3]:
import os
import requests

# Set up Azure OpenAI credentials
api_key = "FPMNN73KO2lW7hWmdTT5DlpZY8YSeC88VQIqmSk5xDy5JDYWpt4wJQQJ99BKACYeBjFXJ3w3AAAAACOGs5Iw"
endpoint = "https://hks-demo-new.cognitiveservices.azure.com/"
deployment_name = "gpt-3.5-turbo"  # Replace with your deployment name

# Create the full endpoint URL
url = f"{endpoint}openai/deployments/{deployment_name}/chat/completions?api-version=2024-10-01-preview"


# Set up headers
headers = {
    "Content-Type": "application/json",
    "api-key": api_key
}


In [4]:
def rag_pipeline(query):
    # Step 1: Retrieve the most relevant document
    retrieved_docs = retrieve(query, top_n=1)
    context = " ".join(retrieved_docs)
    print("Retrieved Document:", context)

    # Step 2: Use Azure OpenAI to generate an answer using the retrieved document
    prompt = f"Based on the following context, answer the question: {query}\n\nContext: {context}"

    # Create the payload
    payload = {
      "messages":[{
          "role":"system",
          "content":"You are an AI assistant that helps people find information.  Only provide answer based on the context that is provided"
        },{
          "role":"user",
          "content":prompt
      }],
      "max_tokens": 100,
      "temperature": 0.7
    }



    # Send the request to Azure OpenAI API
    print(f"Sending request to {url}")
    response = requests.post(url, headers=headers, json=payload)

    if response.status_code == 200:
        result = response.json()
        # return result['choices'][0]['text'].strip()
        return result['choices'][0]['message']['content'].strip()
    else:
        return f"Error: {response.status_code}, {response.text}"


In [5]:
response = rag_pipeline("What is the capital of France?")
print("Final Response:", response)


Retrieved Document: The capital of France is Paris.
Sending request to https://hks-demo-new.cognitiveservices.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2024-10-01-preview


ConnectionError: HTTPSConnectionPool(host='hks-demo-new.cognitiveservices.azure.com', port=443): Max retries exceeded with url: /openai/deployments/gpt-35-turbo/chat/completions?api-version=2024-10-01-preview (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x106642d90>: Failed to resolve 'hks-demo-new.cognitiveservices.azure.com' ([Errno 8] nodename nor servname provided, or not known)"))